# SQL in Python Assignment

In [1]:
import pandas as pd
from sqlalchemy import *
from getpass import getpass

### Connect to the personal database you created yesterday.
Remember the way the credentials work:
```
user = 'postgres' # Replace with your user, if different
pwd = 'pAsSwOrD' # Replace with your password
host = 'localhost'
port = '5432'
db = 'postgres' # Replace with your database name, if different
```

In [2]:
user = 'postgres' # Replace with your user, if different
pwd = getpass()
host = 'localhost'
port = '5432'
db = 'postgres' # Replace with your database name, if different

········


In [9]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    user, pwd, host, port, db))

### Obtain a list of tables that are in the database.

In [10]:
engine.table_names()

['vehicles', 'houseprices', 'city_fuel_18']

### Obtain a list of all the fields that are in the vehicles table along with the data type of each.

In [5]:
meta = MetaData()
vehicles = Table('vehicles', meta, autoload=True, autoload_with=engine)
fields = [(c.name, c.type) for c in vehicles.columns]
fields_df = pd.DataFrame(fields, columns= ['Name', 'Type'])
fields_df

,Name,Type
0,id,BIGINT
1,make,TEXT
2,model,TEXT
3,year,BIGINT
4,class,TEXT
5,trans,TEXT
6,drive,TEXT
7,cyl,DOUBLE PRECISION
8,displ,DOUBLE PRECISION
9,fuel,TEXT


### Write a SELECT query to retrieve all records from the database and load them into a Pandas data frame.

In [7]:
sql = '''
SELECT * 
FROM vehicles;
'''
vehicles_df = pd.read_sql(sql, con=engine)
vehicles_df

,id,make,model,year,class,trans,drive,cyl,displ,fuel,hwy,cty
0,27550,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4.0,2.5,Regular,17,18
1,28426,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4.0,2.5,Regular,17,18
2,27549,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6.0,4.2,Regular,13,13
3,28425,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6.0,4.2,Regular,13,13
4,1032,AM General,Post Office DJ5 2WD,1985,Special Purpose Vehicle 2WD,Automatic 3-spd,Rear-Wheel Drive,4.0,2.5,Regular,17,16
...,...,...,...,...,...,...,...,...,...,...,...,...
33437,33305,smart,fortwo electric drive convertible,2013,Two Seaters,Automatic (A1),Rear-Wheel Drive,NaN,NaN,Electricity,93,122
33438,34393,smart,fortwo electric drive convertible,2014,Two Seaters,Automatic (A1),Rear-Wheel Drive,NaN,NaN,Electricity,93,122
33439,31065,smart,fortwo electric drive coupe,2011,Two Seaters,Automatic (A1),Rear-Wheel Drive,NaN,NaN,Electricity,79,94
33440,33306,smart,fortwo electric drive coupe,2013,Two Seaters,Automatic (A1),Rear-Wheel Drive,NaN,NaN,Electricity,93,122


### Create a table that contains all Audi vehicles with a city fuel efficiency greater than 18.

In [15]:
sql = '''
CREATE TABLE IF NOT EXISTS city_fuel_18 AS
SELECT * 
FROM vehicles
WHERE cty > 18;
'''
engine.execute(sql)

### Delete all records from the temp table you just created.

In [18]:
sql = '''
DELETE FROM city_fuel_18;
'''
engine.execute(sql)

### Insert records from the vehicles table back into the table you created where the make is Audi and the city fuel efficiency is greater than 20.

In [19]:
sql = """
INSERT INTO city_fuel_18
SELECT *
FROM vehicles
WHERE make = 'Audi' AND cty > 20
"""
engine.execute(sql)

### Update the drive field of the records in the table you created to change any "4-Wheel or All-Wheel Drive" values to "All-Wheel Drive."

In [21]:
sql = """
UPDATE city_fuel_18
SET drive = 'All-Wheel Drive'
WHERE drive = '4-Wheel or All-Wheel Drive'
"""
engine.execute(sql)

### Write a for loop that iterates over the list of makes provided below, selects the top 10 vehicles with the highest city fuel efficiency for each make, and then concatenates all the results together into a single Pandas data frame.

In [54]:
makes = ['Ford', 'Chevrolet', 'Toyota', 'Honda']


sql = """
    SELECT *
    FROM vehicles
    LIMIT 1;
    """
columns = engine.execute(sql).fetchall()[0].keys()
df = pd.DataFrame(columns=columns)

for make in makes:
     sql = f"""
     SELECT *
     FROM vehicles
     WHERE MAKE = '{make}'
     ORDER BY cty DESC
     LIMIT 10;
     """
     temp_df = pd.read_sql(sql, con=engine)
     df = df.append(temp_df, ignore_index=True)
    
df

,id,make,model,year,class,trans,drive,cyl,displ,fuel,hwy,cty
0,33024,Ford,Focus Electric,2013,Compact Cars,Automatic (A1),Front-Wheel Drive,NaN,NaN,Electricity,99,110
1,34130,Ford,Focus Electric,2014,Compact Cars,Automatic (A1),Front-Wheel Drive,NaN,NaN,Electricity,99,110
2,32278,Ford,Focus BEV FWD,2012,Compact Cars,Automatic (variable gear ratios),Front-Wheel Drive,NaN,NaN,Electricity,99,110
3,30965,Ford,Ranger Pickup 2WD,2001,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NaN,NaN,Electricity,54,62
4,30966,Ford,Ranger Pickup 2WD,2000,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NaN,NaN,Electricity,54,62
5,30971,Ford,Ranger Pickup 2WD,1999,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NaN,NaN,Electricity,54,62
6,30967,Ford,Ranger Pickup 2WD,2000,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NaN,NaN,Electricity,52,59
7,30972,Ford,Ranger Pickup 2WD,1999,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NaN,NaN,Electricity,44,50
8,33083,Ford,Fusion Hybrid FWD,2013,Midsize Cars,Automatic (variable gear ratios),Front-Wheel Drive,4.0,2.0,Regular,47,47
9,34312,Ford,Fusion Hybrid FWD,2014,Midsize Cars,Automatic (variable gear ratios),Front-Wheel Drive,4.0,2.0,Regular,47,47
